# Original Dataset - ML Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

import sqlalchemy
from sqlalchemy import create_engine, inspect

import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

import numpy as np
from joblib import dump, load
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

In [3]:
# Check for untitled_project folder
if os.path.exists("untitled_project"):
    input("DELETE untitled_project")

## Import datasets

In [4]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'chroma1',
 'chroma10',
 'chroma11',
 'chroma12',
 'chroma2',
 'chroma3',
 'chroma4',
 'chroma5',
 'chroma6',
 'chroma7',
 'chroma8',
 'chroma9',
 'chromastd',
 'delta',
 'demographic',
 'diagnosis',
 'energy',
 'energyentropy',
 'gval',
 'habits',
 'mfcc1',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'rval',
 'spectralcentroid',
 'spectralentropy',
 'spectralflux',
 'spectralrolloff',
 'spectralspread',
 'zcr']

In [5]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [6]:
# Merge dataframes
merged_df = pd.merge(
    dataframes['demographic_df'],
    dataframes['diagnosis_df'],
    how = 'inner',
    on = 'id'
)

merged_df = pd.merge(
    merged_df,
    dataframes['habits_df'],
    how = 'inner',
    on = 'id'
)

# Display merged_df
merged_df.head()

,id,age,gender,occupation_status,diagnosis,subtype,vhi_score,rsi_score,reflux_indicated,vhi_zscore,...,chocolate,chocolate_grams_pd,coffee,coffee_pd,citrus_fruits,citrus_fruits_pd,soft_cheese,soft_cheese_pd,tomatoes,water_litres_pd
0,voice100,24,m,unknown,healthy,no subtype,0,5,0,-0.58,...,sometimes,30,always,3,never,0.00,almost always,100,never,1.5
1,voice101,60,m,unknown,healthy,no subtype,80,10,0,4.76,...,sometimes,30,always,4,never,0.00,sometimes,100,sometimes,1.5
2,voice192,22,m,cook,hyperkinetic dysphonia,no subtype,0,10,0,-0.58,...,always,14,always,3,almost always,1.17,sometimes,100,sometimes,2.5
3,voice193,46,f,housewife,hyperkinetic dysphonia,no subtype,0,36,1,-0.58,...,sometimes,30,always,2,sometimes,1.00,sometimes,100,sometimes,1.0
4,voice008,51,f,researcher,reflux laryngitis,no subtype,19,15,1,0.68,...,almost always,20,always,2,almost always,1.00,sometimes,100,almost always,1.0


## Preprocessing

### Separate the target and feature variables

In [7]:
# Drop the 'id' column
no_id_df = merged_df.drop(columns=['id'])

In [8]:
# Define the target variables
target_var = ['diagnosis', 'subtype']
y = no_id_df[target_var]

# Define the feature variables
X = no_id_df.drop(columns=target_var)

### Binary classification - `diagnosis`

In [9]:
# Encode the target variable, ignore subtype
y = y['diagnosis'].apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Bin `occupation_status` column

In [10]:
# Use limit_unique() function to bin the column
limit_unique(X, 10, ['occupation_status'])

occupation_status
unknown               41
researcher            41
other                 25
employee              25
housewife             23
student               16
technical operator    12
singer                10
pensioner              6
doctor                 5
Name: count, dtype: int64
Number of unique values: 10



### Encode feature columns

#### Encoding
- `smoker` column
	- `0` for `no`
	- `1` for `casual`
	- `2` for `yes`
- `alcohol_consumption` column
	- `0` for `nondrinker`
	- `1` for `casual`
	- `2` for `habitual`
- `carbonated_beverages`, `tomatoes`, `coffee`, `chocolate`, `soft_cheese`, `citrus_fruits` columns
	- `0` for `never`
	- `1` for `almost never`
	- `2` for `sometimes`
	- `3` for `almost always`
	- `4` for `always`

In [11]:
# Define the maps
smoker_map = {
    'no': 0,
    'casual': 1,
    'yes': 2
}

alcohol_map = {
    'nondrinker': 0,
    'casual': 1,
    'habitual': 2
}

habit_map = {
    'never': 0,
    'almost never': 1,
    'sometimes': 2,
    'almost always': 3,
    'always': 4
}

In [12]:
# Apply the label encoding using the maps
X['smoker'] = X['smoker'].map(smoker_map)
X['alcohol_consumption'] = X['alcohol_consumption'].map(alcohol_map)

# Habit columns
habit_cols = [
    'carbonated_beverages', 'tomatoes',
    'coffee', 'chocolate',
    'soft_cheese', 'citrus_fruits'
]

# Use a loop for the habit columns
for habit in habit_cols:
    X[habit] = X[habit].map(habit_map)

In [13]:
# Remove the occupation_status column
# X = X.drop(columns=['occupation_status'])

In [14]:
# Encode the categorical columns using get_dummies
categorical_hot = ['gender', 'occupation_status']
# categorical_hot = ['gender']

# One-hot encoding
encoded_columns = pd.get_dummies(X[categorical_hot]).astype(int)

# Update the feature dataframe
X.drop(categorical_hot, axis=1, inplace=True)
X = pd.concat([X, encoded_columns], axis=1)

# Convert the 'occupation_status_unknown' to 'occupation_status_other'
X.loc[X['occupation_status_unknown'] == 1, 'occupation_status_other'] = 1

# Drop the 'occupation_status_unknown' column
X = X.drop(columns=['occupation_status_unknown'])

In [15]:
# Display the dataframe
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   204 non-null    int64  
 1   vhi_score                             204 non-null    int64  
 2   rsi_score                             204 non-null    int64  
 3   reflux_indicated                      204 non-null    int64  
 4   vhi_zscore                            204 non-null    float64
 5   vhi_impact                            204 non-null    int64  
 6   alcohol_consumption                   204 non-null    int64  
 7   alcohol_pd                            204 non-null    float64
 8   smoker                                204 non-null    int64  
 9   cigarettes_pd                         204 non-null    int64  
 10  carbonated_beverages                  204 non-null    int64  
 11  carbonated_pd      

,age,vhi_score,rsi_score,reflux_indicated,vhi_zscore,vhi_impact,alcohol_consumption,alcohol_pd,smoker,cigarettes_pd,...,gender_m,occupation_status_doctor,occupation_status_employee,occupation_status_housewife,occupation_status_other,occupation_status_pensioner,occupation_status_researcher,occupation_status_singer,occupation_status_student,occupation_status_technical operator
0,24,0,5,0,-0.58,0,1,0.36,0,0,...,1,0,0,0,1,0,0,0,0,0
1,60,80,10,0,4.76,4,0,0.00,0,0,...,1,0,0,0,1,0,0,0,0,0
2,22,0,10,0,-0.58,0,0,0.00,0,0,...,1,0,0,0,1,0,0,0,0,0
3,46,0,36,1,-0.58,0,1,0.36,2,15,...,0,0,0,1,0,0,0,0,0,0
4,51,19,15,1,0.68,1,1,0.36,0,0,...,0,0,0,0,0,0,1,0,0,0


### Split and Scale

In [16]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [17]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Save the scaler
dump(X_scaler, '../voice_app/assets/scaler.joblib')

['../voice_app/assets/scaler.joblib']

## Hyperparameter Tuning

In [19]:
# Define the model parameters
number_input_features = len(X_train.columns)

# Maximum hidden layers (min. 2 for DL)
max_hidden_layers = 5

# Maximum neurons per hidden layer
max_num_neurons = number_input_features * 2 - 1

# Step count
step_count = 5

# Hidden layer activation functions
activation_functions = [
    'relu', 'leaky_relu', 'tanh',
    'elu', 'selu', 'exponential',
    'softmax', 'softplus'
]

# Define the output layer
output_layer_neurons = 1
output_layer_activation = "sigmoid"

# Model compilation
compile_loss = "binary_crossentropy"
compile_opt = "adam"

# Number of max epochs
tuner_max_epochs = 25
search_max_epochs = 25

# Hyperband iterations
hp_iterations = 2

# Regularizers
reg_kernel = regularizers.L1(0.01)

In [20]:
# Extract class name and parameter value, for performance tracker
class_name = reg_kernel.__class__.__name__

if class_name == "L1L2":
    value_l1 = reg_kernel.get_config()['l1']
    value_l2 = reg_kernel.get_config()['l2']

    # Create string version
    reg_kernel_string = f"{class_name}(l1={value_l1:.3f})(l2={value_l2:.3f})"

else:
    param_value = reg_kernel.get_config()[f'{class_name.lower()}']

    # Create string version
    reg_kernel_string = f"{class_name}({param_value:.3f})"

print(reg_kernel_string)

L1(0.010)


In [21]:
# Initialise the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = tuner_max_epochs,
    hyperband_iterations = hp_iterations
)

In [22]:
# Find the best hyperparameters
tuner.search(
    X_train_scaled,
    y_train,
    epochs = search_max_epochs,
    validation_data = (X_test_scaled, y_test)
)

Trial 60 Complete [00h 00m 01s]
val_accuracy: 0.7254902124404907

Best val_accuracy So Far: 0.8235294222831726
Total elapsed time: 00h 00m 33s


## Compile, Train, Evaluate the Best Model

In [23]:
# Get the top 3 model hyperparameters
top3_hyper = tuner.get_best_hyperparameters(3)

for model in top3_hyper:
    pprint(model.values)

{'activation_layer_0': 'selu',
 'activation_layer_1': 'elu',
 'activation_layer_2': 'exponential',
 'activation_layer_3': 'elu',
 'activation_layer_4': 'softmax',
 'num_layers': 3,
 'tuner/bracket': 0,
 'tuner/epochs': 25,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_layer_0': 41,
 'units_layer_1': 41,
 'units_layer_2': 26,
 'units_layer_3': 41,
 'units_layer_4': 51}
{'activation_layer_0': 'relu',
 'activation_layer_1': 'selu',
 'activation_layer_2': 'selu',
 'activation_layer_3': 'tanh',
 'activation_layer_4': 'elu',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 25,
 'tuner/initial_epoch': 9,
 'tuner/round': 2,
 'tuner/trial_id': '0015',
 'units_layer_0': 21,
 'units_layer_1': 21,
 'units_layer_2': 56,
 'units_layer_3': 36,
 'units_layer_4': 6}
{'activation_layer_0': 'relu',
 'activation_layer_1': 'selu',
 'activation_layer_2': 'selu',
 'activation_layer_3': 'tanh',
 'activation_layer_4': 'elu',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 9,
 'tuner/init

In [24]:
# Get the top model
best_hyper = top3_hyper[0]
best_hyper.values

{'activation_layer_0': 'selu',
 'units_layer_0': 41,
 'num_layers': 3,
 'units_layer_1': 41,
 'activation_layer_1': 'elu',
 'units_layer_2': 26,
 'activation_layer_2': 'exponential',
 'units_layer_3': 41,
 'activation_layer_3': 'elu',
 'units_layer_4': 51,
 'activation_layer_4': 'softmax',
 'tuner/epochs': 25,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [25]:
# Define the model parameters
number_input_features = len(X_train.columns)
hidden_layer0_neurons = best_hyper.values['units_layer_0']
hidden_layer0_activation = best_hyper.values['activation_layer_0']

# Return the number of hidden layers
total_hidden = best_hyper.values['num_layers'] + 1

# Train model
model_train_epochs = 100

In [26]:
# Initialise the sequential model
nn = Sequential()

# Create the input layer and first hidden layer
nn.add(Dense(
    units = hidden_layer0_neurons,
    activation = hidden_layer0_activation,
    kernel_regularizer = reg_kernel,
    input_dim = number_input_features
))
        
# Create additional hidden layers
for layer in range(1, total_hidden):
    nn.add(Dense(
        units = best_hyper.values[f'units_layer_{layer}'],
        activation = best_hyper.values[f'activation_layer_{layer}'],
        kernel_regularizer = reg_kernel
    ))

# Create the output layer
nn.add(Dense(
    units = output_layer_neurons,
    activation = output_layer_activation
))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 41)                1394      
                                                                 
 dense_7 (Dense)             (None, 41)                1722      
                                                                 
 dense_8 (Dense)             (None, 26)                1092      
                                                                 
 dense_9 (Dense)             (None, 41)                1107      
                                                                 
 dense_10 (Dense)            (None, 1)                 42        
                                                                 
Total params: 5357 (20.93 KB)
Trainable params: 5357 (20.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(
    loss = compile_loss,
    optimizer = compile_opt,
    metrics = ["accuracy"]
)

In [28]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    shuffle = True,
    epochs = model_train_epochs,
    verbose = 1
)

Epoch 1/100
5/5 [==============================] - 0s 997us/step - loss: 8.1581 - accuracy: 0.6209
Epoch 2/100
5/5 [==============================] - 0s 828us/step - loss: 7.8547 - accuracy: 0.6993
Epoch 3/100
5/5 [==============================] - 0s 822us/step - loss: 7.6583 - accuracy: 0.7647
Epoch 4/100
5/5 [==============================] - 0s 784us/step - loss: 7.4984 - accuracy: 0.7974
Epoch 5/100
5/5 [==============================] - 0s 847us/step - loss: 7.3438 - accuracy: 0.7974
Epoch 6/100
5/5 [==============================] - 0s 885us/step - loss: 7.1981 - accuracy: 0.8039
Epoch 7/100
5/5 [==============================] - 0s 831us/step - loss: 7.0497 - accuracy: 0.8039
Epoch 8/100
5/5 [==============================] - 0s 909us/step - loss: 6.9046 - accuracy: 0.8105
Epoch 9/100
5/5 [==============================] - 0s 879us/step - loss: 6.7612 - accuracy: 0.8105
Epoch 10/100
5/5 [==============================] - 0s 847us/step - loss: 6.6165 - accuracy: 0.8170
Epoch 11/

## Evaluate the Model Results

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.8753 - accuracy: 0.8039 - 48ms/epoch - 24ms/step
Loss: 0.8752579689025879, Accuracy: 0.8039215803146362


In [30]:
# Check the prediction's output probabilities
predicted_prob = nn.predict(X_test_scaled)
clean_prob = np.round(predicted_prob, 2)

# Round to the nearest integer and flatten
clean_predicted = np.round(predicted_prob).astype(int).flatten()

# Convert to a dataframe for readability
output_prob = pd.DataFrame({
    'Actual': y_test,
    'Predicted': clean_predicted,
    'Probability': clean_prob.flatten()
})

output_prob.head(10)

2/2 [==============================] - 0s 1ms/step


,Actual,Predicted,Probability
99,1,1,0.89
62,1,1,0.94
188,1,1,0.89
195,1,1,0.93
97,0,0,0.06
32,1,1,0.89
144,0,0,0.46
126,1,1,0.95
50,1,1,0.86
107,1,1,0.91


In [31]:
# Display the confusion matrix
cmatrix = confusion_matrix(y_test, clean_predicted)

# Convert the matrix to a DataFrame
cmatrix_df = pd.DataFrame(
    cmatrix,
    index = ["Actual 0", "Actual 1"],
    columns = ["Predicted 0", "Predicted 1"]
)

# Display the confusion matrix
print("Confusion Matrix:")
cmatrix_df

Confusion Matrix:


,Predicted 0,Predicted 1
Actual 0,6,8
Actual 1,2,35


In [32]:
# Print the classification report for the model
creport = classification_report(
    y_test,
    clean_predicted,
    target_names = ["Healthy (0)", "Pathological (1)"]
)

print("Classification Report:\n", creport)

Classification Report:
                   precision    recall  f1-score   support

     Healthy (0)       0.75      0.43      0.55        14
Pathological (1)       0.81      0.95      0.88        37

        accuracy                           0.80        51
       macro avg       0.78      0.69      0.71        51
    weighted avg       0.80      0.80      0.78        51



In [33]:
# Extract the precision and recall
precision = precision_score(y_test, clean_predicted, average=None)
recall = recall_score(y_test, clean_predicted, average=None)
print(precision, recall)

[0.75       0.81395349] [0.42857143 0.94594595]


## Save Results to Performance Tracker

In [34]:
# Create dictionary to save the results
results_dict = dict()

current_time = datetime.now()
results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_loss'] = model_loss
results_dict['model_accuracy'] = model_accuracy
results_dict['precision_0'] = precision[0]
results_dict['precision_1'] = precision[1]
results_dict['recall_0'] = recall[0]
results_dict['recall_1'] = recall[1]

In [35]:
# Print the model architecture
print(f"Input features: {number_input_features}")
print(f"Hidden Layer 0: {hidden_layer0_activation}, {hidden_layer0_neurons}")

# Populate results_dict with model details
results_dict['input_features'] = number_input_features
results_dict['num_layers'] = best_hyper.values['num_layers']
results_dict['hlayer_0_activation'] = hidden_layer0_activation
results_dict['hlayer_0_neurons'] = hidden_layer0_neurons
results_dict['kernel_regularizer'] = reg_kernel_string

for layer in range(1, total_hidden):
    activation = best_hyper.values[f'activation_layer_{layer}']
    neurons = best_hyper.values[f'units_layer_{layer}']
    print(f"Hidden Layer {layer}: {activation}, {neurons}")
    
    results_dict[f'hlayer_{layer}_activation'] = best_hyper.values[
        f'activation_layer_{layer}']
    results_dict[f'hlayer_{layer}_neurons'] = best_hyper.values[
        f'units_layer_{layer}']

results_dict['olayer_neurons'] = output_layer_neurons
results_dict['olayer_activation'] = output_layer_activation

# Populate results_dict with tuning details
results_dict['tuning_max_hidden'] = max_hidden_layers
results_dict['tuning_max_neurons'] = max_num_neurons
results_dict['tuning_step_count'] = step_count
results_dict['activation_functions'] = str(activation_functions)
results_dict['tuning_tuner_epochs'] = tuner_max_epochs
results_dict['tuning_search_epochs'] = search_max_epochs
results_dict['tuning_hp_iterations'] = hp_iterations

# Populate results_dict with compilation details
results_dict['compile_loss'] = compile_loss
results_dict['compile_optimizer'] = compile_opt

Input features: 33
Hidden Layer 0: selu, 41
Hidden Layer 1: elu, 41
Hidden Layer 2: exponential, 26
Hidden Layer 3: elu, 41


In [36]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  changed occupation status binning again


In [37]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 6, 11, 25, 21, 353046),
 'runtime': 39.32138800621033,
 'model_loss': 0.8752579689025879,
 'model_accuracy': 0.8039215803146362,
 'precision_0': 0.75,
 'precision_1': 0.813953488372093,
 'recall_0': 0.42857142857142855,
 'recall_1': 0.9459459459459459,
 'input_features': 33,
 'num_layers': 3,
 'hlayer_0_activation': 'selu',
 'hlayer_0_neurons': 41,
 'kernel_regularizer': 'L1(0.010)',
 'hlayer_1_activation': 'elu',
 'hlayer_1_neurons': 41,
 'hlayer_2_activation': 'exponential',
 'hlayer_2_neurons': 26,
 'hlayer_3_activation': 'elu',
 'hlayer_3_neurons': 41,
 'olayer_neurons': 1,
 'olayer_activation': 'sigmoid',
 'tuning_max_hidden': 5,
 'tuning_max_neurons': 65,
 'tuning_step_count': 5,
 'activation_functions': "['relu', 'leaky_relu', 'tanh', 'elu', 'selu', 'exponential', 'softmax', 'softplus']",
 'tuning_tuner_epochs': 25,
 'tuning_search_epochs': 25,
 'tuning_hp_iterations': 2,
 'compile_loss': 'binary_crossentropy',
 'compile_optimizer': 'adam

In [38]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_loss,model_accuracy,precision_0,precision_1,recall_0,recall_1,input_features,num_layers,...,tuning_max_hidden,tuning_max_neurons,tuning_step_count,activation_functions,tuning_tuner_epochs,tuning_search_epochs,tuning_hp_iterations,compile_loss,compile_optimizer,change_message
0,2024-01-06 11:25:21.353046,39.321388,0.875258,0.803922,0.75,0.813953,0.428571,0.945946,33,3,...,5,65,5,"['relu', 'leaky_relu', 'tanh', 'elu', 'selu', ...",25,25,2,binary_crossentropy,adam,changed occupation status binning again


In [39]:
# Performance tracker
tracker_path = "../resources/tracker/performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/run_{len(tracker_df)}_{model_pct}.h5', save_format='h5')

else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/run_0_{model_pct}.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
